In [1]:
# https://colab.research.google.com/github/huggingface/notebooks/blob/main/transformers_doc/en/pytorch/training.ipynb#scrollTo=bHSO-M_0gMrN
%load_ext autoreload
%autoreload 1

In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
dataset["train"][100]

Found cached dataset yelp_review_full (/home/aris/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf)


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [3]:
from transformers import AutoTokenizer

model_name = 'bert-base-cased'

tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/aris/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-abfb413a29e68f59.arrow
Loading cached processed dataset at /home/aris/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-fb3047aca4907541.arrow


In [4]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(
    range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(
    range(1000))

Loading cached shuffled indices for dataset at /home/aris/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-e610d8dc05503c8d.arrow
Loading cached shuffled indices for dataset at /home/aris/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-844145ae27637fd0.arrow


In [5]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                           num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [6]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [7]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [8]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch")

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [11]:
# trainer.train()

In [12]:
import torch

del model
del trainer
torch.cuda.empty_cache()

In [13]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [14]:
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

In [15]:
tokenized_datasets.set_format("torch")

In [16]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(
    range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(
    range(1000))

Loading cached shuffled indices for dataset at /home/aris/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-27b87dd698b38540.arrow
Loading cached shuffled indices for dataset at /home/aris/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf/cache-7b0b6d57740fe4a8.arrow


In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

In [18]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                           num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [19]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [20]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [21]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [22]:
# from tqdm.auto import tqdm
#
# progress_bar = tqdm(range(num_training_steps))
#
# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()
#
#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)

In [23]:
# import evaluate
#
# metric = evaluate.load("accuracy")
# model.eval()
# for batch in eval_dataloader:
#     batch = {k: v.to(device) for k, v in batch.items()}
#     with torch.no_grad():
#         outputs = model(**batch)
#
#     logits = outputs.logits
#     predictions = torch.argmax(logits, dim=-1)
#     metric.add_batch(predictions=predictions, references=batch["labels"])
#
# metric.compute()

In [24]:
from torchinfo import summary

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased",
                                                           num_labels=5)
model.to(device)

params = dict(model.named_parameters())
buffers = dict(model.named_buffers())

# Make trainable parameters
params = {k: v for k, v in params.items() if v.requires_grad}

for k, v in model.named_parameters():
    v.requires_grad_(False)

summary(model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Layer (type:depth-idx)                                  Param #
BertForSequenceClassification                           --
├─BertModel: 1-1                                        --
│    └─BertEmbeddings: 2-1                              --
│    │    └─Embedding: 3-1                              (22,268,928)
│    │    └─Embedding: 3-2                              (393,216)
│    │    └─Embedding: 3-3                              (1,536)
│    │    └─LayerNorm: 3-4                              (1,536)
│    │    └─Dropout: 3-5                                --
│    └─BertEncoder: 2-2                                 --
│    │    └─ModuleList: 3-6                             (85,054,464)
│    └─BertPooler: 2-3                                  --
│    │    └─Linear: 3-7                                 (590,592)
│    │    └─Tanh: 3-8                                   --
├─Dropout: 1-2                                          --
├─Linear: 1-3                                           (3,845)
To

In [25]:
from torch.func import functional_call, vmap, grad_and_value
from functools import partial
from grabngo import GraBSampler
%aimport grabngo
import torchopt
from tqdm.auto import tqdm

optimizer = torchopt.adamw(
    lr=torchopt.schedule.linear_schedule(5e-5, 0, num_training_steps),
)
opt_state = optimizer.init(params)


# pure function
def compute_loss(model, params, buffers, kwargs):
    # vamp remove the first dimension, which is batch size, but bert requires it
    kwargs = {k: v.unsqueeze(0) for k, v in kwargs.items()}

    # Gary: BERT doesn't treat input_ids as named kwargs, so we need to
    # explicitly pass it in
    input_ids = kwargs['input_ids']
    kwargs.pop('input_ids')
    out = functional_call(
        module=model,
        parameter_and_buffer_dicts=(params, buffers),
        args=(input_ids,),
        kwargs=kwargs,
    )

    return out.loss


def get_func(batch: dict):
    return vmap(
        grad_and_value(partial(compute_loss, model)),
        in_dims=(None, None, {k: 0 for k in batch.keys()}),
    )  # the only argument of compute_loss is batched along the first axis


sampler = GraBSampler(
    data_source=small_train_dataset,
    trainable_params=params,
    balance_type='mean',
)

train_dataloader = DataLoader(small_train_dataset, sampler=sampler,
                              batch_size=2)

# Compute per sample gradients and loss
print(next(iter(small_train_dataset)).keys())
ft_compute_sample_grad_and_loss = get_func(next(iter(small_train_dataset)))

process_bar = tqdm(range(num_training_steps))

for epoch in range(3):
    for _, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        # print(batch.keys())
        ft_per_sample_grads, batch_loss = ft_compute_sample_grad_and_loss(
            params, buffers, batch
        )

        demo_layer = ft_per_sample_grads[list(params.keys())[0]]
        print(demo_layer.shape)

        #######################################################################
        sampler.step(ft_per_sample_grads)  # <- step compute GraB algorithm
        #######################################################################

        # The following is equivalent to
        # optimizer.zero_grad()
        # loss.backward()
        # optimizer.step()
        grads = {k: g.mean(dim=0) for k, g in ft_per_sample_grads.items()}
        updates, opt_state = optimizer.update(
            grads, opt_state, params=params
        )  # get updates
        params = torchopt.apply_updates(
            params, updates
        )  # update model parameters
        process_bar.update(1)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:40                                                                                   │
│                                                                                                  │
│   37 │   )  # the only argument of compute_loss is batched along the first axis                  │
│   38                                                                                             │
│   39                                                                                             │
│ ❱ 40 sampler = GraBSampler(                                                                      │
│   41 │   data_source=small_train_dataset,                                                        │
│   42 │   params=params,                                                                          │
│   43 │   balance_type='mean',                                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: GraBSampler.__init__() missing 1 required positional argument: 'trainable_params'

In [ ]:
 import evaluate

metric = evaluate.load("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()